**Important notes:** This notebook pipeline only works with protein sequences (no headers) or uploads of zipped fasta files (with headers). To upload zipped fasta files, click the "File" icon on the left panel, and click the "Upload to session storage" icon. These files will be erased at the end of each session, or after the time limit imposed by Google Colaboratory (~12 hours).

In [ ]:
#@title 1. Run pipeline:
#@markdown 1) First, set inputs (3. Set inputs).<br>
#@markdown 1b) If you selected custom motif options, follow the prompt to upload files. <br>
#@markdown 2) Next, click and highlight this cell (1. Run pipeline). <br>
#@markdown 3) Click "Runtime" from the top menu and "Run all" <br> 

In [ ]:
#@title <- Install dependencies before running the pipeline. (~3-6min)
# install conda package manager
# For more information: https://docs.conda.io/en/latest/
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -y -c bioconda meme=4.11.2 &>/dev/null #5.3.0
!conda install -y -c conda-forge biopython &>/dev/null

In [ ]:
#@title 2. Set inputs

from google.colab import files
import os.path
import re
import hashlib
import random
import glob2
import pandas as pd
from time import perf_counter
import argparse
import sys

def add_hash(x,y):
  '''
  Give jobname unique identifier
  '''
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

# Acquire user input query sequence
query_sequence = '' #@param {type:"string"}

# Regardless of whether you've input a sequence or not
# Acquire boolean (checkbox) value for zipped fasta
Upload_zipped_fastas = True #@param {type:"boolean"}

# Acquire user input jobname
jobname = 'primate_seqs' #@param {type:"string"}

# Acquire user selected drop-down motif option
motif_options = "custom" #@param ["none", "E75","custom"]
#@markdown * Warning: Do not select more than one motif upload.

# Acquire user input pval number
pval = 0.00581 #@param {type:"number"}

# Acquire boolean check for pos. sel. residue (PSR) files
enable_PSRs = True #@param {type:"boolean"}

# Acquire boolean check for prebuilt db .csv
enable_prebuiltdb = True #@param {type:"boolean"}

is_input = False
query_sequence = "".join(re.split('; |, |\*|\n|-| ', query_sequence)).upper() # remove whitespaces
if query_sequence:
  is_input = True
  non_AAs = {'O', 'U', 'X', 'B', 'Z', 'J'}
  if non_AAs.intersection(query_sequence): 
    print("Warning: Invalid AA(s) in sequence.")
else:
  query_sequence = "empty"

if Upload_zipped_fastas:
  seq_zipfiles = glob2.glob("*seqs.zip")
  if seq_zipfiles:
    is_input = True
    print("Zipped file(s) found:", *seq_zipfiles)
  else:
    print("Zipped file(s) not found. Try reuploading with a seqs.zip suffix.")

if is_input:
  # Acquire user input jobname and add unique identifier
  basejobname = "".join(jobname.split()) # remove whitespaces
  basejobname = re.sub(r'\W+', '', basejobname)
  jobname = add_hash(basejobname, query_sequence)
  while os.path.isfile(f"{jobname}.csv"):
    jobname = add_hash(basejobname, ''.join(random.sample(query_sequence,len(query_sequence))))
  with open(f"{jobname}.csv", "w") as text_file:
    text_file.write(f"id,sequence\n{jobname},{query_sequence}")
  print("Setting up job:", jobname)

  # Placeholder - this will be the final output
  os.mkdir(jobname)
  # queries_path=f"{jobname}/{jobname}.csv"
else:
  print("No input sequences found.")

# Check for motif input
if motif_options == "E75":
  use_motif = True
  custom_motif_path = None #
elif motif_options == "custom":
  print("Input MEME-formatted motif .txt file:")
  custom_motif_path = f"{jobname}/motif/"
  os.mkdir(custom_motif_path)
  uploaded_motif = files.upload()
  use_motif = True
  motif = ""
  for fn in uploaded_motif.keys():
    os.rename(fn, f"{jobname}/motif/{fn}")
    motif = f"{custom_motif_path}{fn}"
else:
  print("Please select/upload a motif.")
  custom_motif_path = None
  use_motif = False

# check for PSR input
if enable_PSRs:
  PSRs = glob2.glob("*posteriors.zip")
  if PSRs:
    is_PSRs = True
    print("Including PSR file(s) to be mapped.")
  else:
    enable_PSRs = False
    print("PSR file(s) not found. Try reuploading with a posteriors.zip suffix.")

# check for prebuilt db .csv input
if enable_prebuiltdb and query_sequence != "empty":
  print("Input pre-built db.csv file:")
  custom_prebuiltdb_path = f"{jobname}/db/"
  os.mkdir(custom_prebuiltdb_path)
  uploaded_db = files.upload()
  use_db = True
  prebuiltdb = ""
  for fn in uploaded_db.keys():
    os.rename(fn, f"{jobname}/db/{fn}")
    prebuiltdb = f"{custom_prebuiltdb_path}{fn}"
else:
  print("Please select/upload a pre-built db.")
  custom_prebuiltdb_path = None
  use_db = False

In [ ]:
#@title 4. Build sequences and related files
t1_start = perf_counter()

# Make input directory
custom_seq_path = f"{jobname}/seqs"
os.mkdir(custom_seq_path)

# Create .fa for input query and send to directory
if query_sequence != "empty":
  with open(f"{custom_seq_path}/{jobname}.12taxa.fa", 'w') as inputfa:
    inputfa.write(f">{jobname}.12taxa.fa_QuerySeq\n")
    inputfa.write(f"{query_sequence}\n")

# Unzip very quietly (qq) and send .fa files to directory
if Upload_zipped_fastas:
  for seq_zip in seq_zipfiles:
    !unzip -qq -j $seq_zip -d $custom_seq_path

# Unzip very quietly (qq) and send .posterior.csv files to directory
if enable_PSRs:
  custom_PSR_path = f"{jobname}/posteriors"
  os.mkdir(custom_PSR_path)
  for PSR in PSRs:
    !unzip -qq -j $PSR -d $custom_PSR_path
else:
  custom_PSR_path = ""

t1_stop = perf_counter()
print("Elapsed time (seconds):", t1_stop-t1_start)

In [ ]:
#@title 5. Experimental - FIMO Search motif against all inputs (~10min)
#@markdown parallelized across ~2 processes given by colab
t1_start = perf_counter()

# Create directory for FIMO output
print(f"Creating FIMO output directory.")
custom_FIMO_path = f"{jobname}/FIMO_out"
os.mkdir(custom_FIMO_path)
print(f"Writing files to: {custom_FIMO_path}.")

# Create script to run FIMO on every file in specified directory
sh = """
#!/bin/zsh

# "$#" = value of the total number of command line args passed
# As long as "$#" is greater than (-gt) 0 args, keep while loop alive
while [[ "$#" -gt 0 ]]; do
    # Check each case (options/flags) until match is found
    case $1 in
        # get input following arg option, then shift to next str
        -i|--inputdir) inputdir="$2"; shift ;;
        -m|--motif) motif="$2"; shift ;;
        -p|--pval) pval="$2"; shift ;;
        -o|--oc) outputdir="$2"; shift ;;
        
        # if extra, unmatched options show up, exit
        # Exit code 0 - Success
        # Exit code 1 - General/misc errors, such as "divide by zero" and other impermissible operations
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
    
    # end case search (case spelled backwards)
    esac
    shift # to the next str, if any, then loop
done
N=4
(
for f in $inputdir/*.fa
do
    ((i=i%N)); ((i++==0)) && wait
    # get basename of filepath
    baseFname=$(basename $f .fa)
    # FIMO on each file
    fimo --oc "$outputdir" --verbosity 1 --text --thresh $pval --max-stored-scores 8000000 "$motif" "$f" > "$outputdir/$baseFname""_fimo.tsv" &
done
)
"""
with open('parallel_FIMO.sh', 'w') as file:
  file.write(sh)

# Run parallel FIMO script
print(f"Running iterative FIMO...")
!bash parallel_FIMO.sh -i $custom_seq_path -m $motif -p $pval -o $custom_FIMO_path &>/dev/null
print(f"iterative_FIMO completed.")

# Delete empty .tsv files created by FIMO
print(f"Cleaning empty FIMO .tsv files...")
!find $custom_FIMO_path -size 0 -print -delete &>/dev/null
print(f"Directory cleaned.")

t1_stop = perf_counter()
print("Elapsed time (seconds):", t1_stop-t1_start)

In [ ]:
#@title 5. FIMO Search motif against all inputs (~10min)
#@markdown this is too slow -- parallelize it

# Create directory for FIMO output
print(f"Creating FIMO output directory.")
custom_FIMO_path = f"{jobname}/FIMO_out"
os.mkdir(custom_FIMO_path)
print(f"Writing files to: {custom_FIMO_path}.")

# Run FIMO on every file in specified directory
print(f"Running iterative FIMO...")
for in_fa in glob2.glob(f"{custom_seq_path}/*.fa"):
  # get basename of filepath
  baseFname = in_fa.split(".")[0].split("/")[-1]
  # FIMO on each file
  !fimo --oc $custom_FIMO_path --verbosity 1 --text --thresh $pval --max-stored-scores 8000000 \
  $motif $in_fa > "$custom_FIMO_path/$baseFname""_fimo.tsv" 2> /dev/null
print(f"iterative_FIMO completed.")

# Delete empty .tsv files created by FIMO
print(f"Cleaning empty FIMO .tsv files...")
!find $custom_FIMO_path -size 0 -print -delete
print(f"Directory cleaned.")

In [ ]:
#@title 6. Summarize all FIMO .tsv inputs (~15min)

from dataclasses import dataclass
from typing import List

@dataclass
class arg_collection:
    csv_out: str
    fimo_tsvs: List[str]
    fasta_dir: str
    PSG_dir: str
    def __post_init__(self):
        if not self.PSG_dir:
            self.PSG_dir = 'skip'

def glob_files(path: str) -> List[str]:
    return(glob2.glob(f'{path}/*.tsv'))

def expand_motif_aln(aln_dir: str, in_seqID: str, seq_sites: List[int]) -> (int, List[str]):
    '''Default shows only the species sequences with motif hits.
    When an aln dir is specified, this function is invoked to
    show the motif alignment across both hits and nonhits.'''
    from Bio.SeqIO.FastaIO import SimpleFastaParser

    # collect species name from title and relevant motif alignment from sequence
    with open(f'{aln_dir}/{in_seqID}.12taxa.fa') as aln_file:
        species_regions = []
        seq_length = 0
        for title, sequence in SimpleFastaParser(aln_file):
            species_name = title.split('_')[-1]
            if species_name == 'hg38' or species_name == 'QuerySeq':
                seq_length = len(sequence.replace('-',''))
            species_regions.append([f'{species_name}: {sequence[pos-1:pos+7]}' for pos in seq_sites])
    return(seq_length, list(map(list, zip(*species_regions))))

def map_PSRs(PSG_dir: str, in_seqID: str, seq_sites: List[int]) -> List[List[str]]:
    '''Returns stringmap of Positive Selection at Residues (PSRs) from dir of FUBAR files, 
    if relevant to the motif range (pos-1:pos+7). PSRs are recorded as '+', 
    and non-PSRs are recorded as '-'.'''
    site_map = [list(range(pos-1,pos+7)) for pos in seq_sites]
    PSRs = []
    try: # File may or may not exist, but if it does, collect PSR entries
        with open(f'{PSG_dir}/{in_seqID}.12taxa.fa.12taxon.tree.grid_info.posteriors.csv') as PSG_file:
            next(PSG_file) # Skip first line
            for line in PSG_file:
                PSR = int(line.split('0.')[0])
                PSRs.append(PSR)
        
        # map sites with presence '+' or absence '_' of PSR
        for site_i, site in enumerate(site_map):
            for pos_i, pos in enumerate(site):
                if pos in PSRs:
                    site_map[site_i][pos_i] = '+'
                else:
                    site_map[site_i][pos_i] = '-'
            site_map[site_i] = ''.join(site)
        return(site_map)
    except IOError: #if file doesn't exist, return default string
        return(['--------' for _ in range(len(seq_sites))])
    except StopIteration: #if file exists, but there are no sites, return default string
        return(['--------' for _ in range(len(seq_sites))])

def exclude_hits(hits_to_exclude: List[str], all_hits: List[str]) -> List[str]:
    nonhit_regions = []
    for hit_index, species in enumerate(hits_to_exclude):
        nonhit_regions.append(set(all_hits[hit_index]).symmetric_difference(species))
    return(nonhit_regions)

# pandas .agg func rename
def Num_Unique(series: pd.Series) -> int:
    return(len(set(series)))

def human_hit(series: pd.Series) -> str:
    if series.str.contains('hg38').any():
        return('Yes')
    else:
        return('No')

def main():
    """
    Takes fimo files and aln files (optional) and generates
    summary dataframe containing one motif hit per line
    with the following info: 
    seq ID|AA pos|species hit: seq|min pval|hg38? (yes/no)|hg38 site|hg38 pval|species absent
    """
    t1_start = perf_counter()

    args = arg_collection(
        f"{jobname}/{jobname}.csv",
        custom_FIMO_path,
        f"{custom_seq_path}",
        custom_PSR_path
    )

    # Check if this file already exists. If so, do nothing.
    if os.path.isfile(args.csv_out):
        sys.exit(f"File {args.csv_out} exists")
    else:
        print (f"Building {args.csv_out}...")

    #Set files and columns to extract from
    infimo_files = glob_files(args.fimo_tsvs.rstrip('/'))
    infile_ind = [1, 2, 6, 8] # 'sequence name', 'start', 'p-value', 'matched sequence'
    agg_func_text = {'seqIDs': ['first'], # get one representative seqID (first occurrence)
                    'start': ['first', 'count'], # get representative start val, and count of species hits
                    'species_seqs': [tuple], # summarize species seq hits as tuple
                    'matchedseq': [Num_Unique], # num of unique seq hits found
                    'species_pvals': [tuple], # scores for each species hit
                    'pvalue': 'min', # best hit, no matter what species
                    'species': [human_hit]} # Is this a human hit? Yes or No

    for ind, file in enumerate(infimo_files):
        # Create dataframe with selected data from fimo file
        tsv_data = pd.read_csv(file, sep = '\t', usecols = infile_ind, 
                                names = ['seqname', 'start', 'pvalue', 'matchedseq'])
        
        # Temporary hack, not intended to have .12taxa.fa_
        tsv_data[['seqIDs', 'species']] = tsv_data.seqname.str.split('.12taxa.fa_', expand=True)
        tsv_data['species_seqs'] = tsv_data['species'].astype(str) + ': ' + tsv_data['matchedseq']
        tsv_data['species_pvals'] = tsv_data['species'].astype(str) + ': ' + tsv_data['pvalue'].astype(str)

        #Retain unmerged data: hg38 matchedseq and pvalue data
        hg_data = tsv_data[tsv_data['species'] == 'hg38'].sort_values('start', axis = 0, ascending = True)

        #collapse tsv_data to one line per motif hit across orgs
        tsv_data = (tsv_data.iloc[1: , 1:]
                    .groupby(['seqIDs', 'start'], as_index = False)
                    .agg(agg_func_text))

        # hard-coded -- this order doesn't need to change
        tsv_data.columns = ['sequenceID', 'start', 'count', 'concat_sites', 'Num_Unique', 'org_pvals', 'best_pval', 'human_hit'] # replace w/ readable colnames 

        #merge tsv_data to retained hg38 data and export
        merged_data = pd.merge(tsv_data, hg_data[['start', 'matchedseq', 'pvalue']],on='start', how='left')

        #use seqID, start pt, and species hits to scrape sequences of orgs with no detectable motif
        #collect species-relevant motif info
        aln_directory = args.fasta_dir.rstrip('/')
        grp_seqID = tsv_data['sequenceID'][0]
        mstarts = tsv_data.start.astype(int) #motif start sites
        sp_hits_to_exclude = tsv_data.concat_sites

        #extract protein (AA) seq length, [aln of each motif across all primates] regardless of score
        AA_length, sp_mregions = expand_motif_aln(aln_directory, grp_seqID, mstarts)

        #exclude hits already examined
        nonhit_mregions = exclude_hits(sp_hits_to_exclude, sp_mregions)
        
        #create nonhit df to merge
        nonhit_df = pd.DataFrame(columns = ['start', 'Non_hits'])
        nonhit_df['Non_hits'] = nonhit_mregions
        nonhit_df['start'] = tsv_data.start
        nonhit_df['AA_seqlength'] = AA_length

        #merge nonhit df to merged_data
        merged_data = pd.merge(merged_data, nonhit_df[['start', 'Non_hits', 'AA_seqlength']],on='start', how='left')

        #OPTIONAL: use seqID and start pt to scrape residues under pos sel (PSRs)
        if args.PSG_dir != 'skip':
            #collect PSG relevant info
            PSG_directory = args.PSG_dir.rstrip('/')
            grp_seqID = tsv_data['sequenceID'][0]
            mstarts = tsv_data.start.astype(int) #motif start sites

            #extract protein (AA) seq length, [aln of each motif across all primates] regardless of score
            PSR_stringmap = map_PSRs(PSG_directory, grp_seqID, mstarts)

            #create nonhit df to merge
            PSR_df = pd.DataFrame(columns = ['start', 'PSRs'])
            PSR_df['start'] = tsv_data.start
            PSR_df['FUBAR_PSRs'] = PSR_stringmap

            #merge nonhit df to merged_data
            merged_data = pd.merge(merged_data, PSR_df[['start', 'FUBAR_PSRs']],on='start', how='left')

        #Create csv file if first glob file initiated, otherwise append to existing csv
        if ind == 0:
            merged_data.rename(columns={'matchedseq': 'human_site', 'pvalue': 'pval_hg38'}, inplace=True)
            merged_data.to_csv(args.csv_out, index = False, mode = 'w', header=True)
        else:
            merged_data.to_csv(args.csv_out, index = False, mode = 'a', header=False)

    t1_stop = perf_counter()
    print("Elapsed time (seconds):", t1_stop-t1_start)

main()

In [ ]:
#@title Create orthogonal dataset .csv (<1min)

import argparse
import glob
import pandas as pd

def parse_args():
    parser=argparse.ArgumentParser(prog='merge-dbs.py', conflict_handler='resolve')
    parser.add_argument('-db', type=str, required=True, help='=> path/to/main_db.csv')
    parser.add_argument('-db_dir', type=str, required=True, help='=> path/to/database_directory')
    parser.add_argument('-o', type=str, required=True, help='=> path/to/merged_outfile.csv')
    return(parser.parse_args())

def glob_files(path: str) -> list[str]:
    return(glob.glob(f'{path}/*'))

def auto_merge(in_df: pd.DataFrame, glob_files: list[str]) -> tuple[pd.DataFrame, list[str]]:
    leftovers=[]
    for file in glob_files:
        # Create dataframe with selected data from each compatible db files in loop
        filename=file.split('/')[-1]
        db=pd.read_csv(file)
        cols=list(db)
        col_intersect=list(in_df.columns.intersection(db.columns)) # get key col, assume one

        # If shared key col exists, unpack to unique var, else skip/report file error
        if col_intersect:
            cols.insert(0, cols.pop(cols.index(col_intersect[0]))) # move intersected key col to the front
            db=db.loc[:, cols]
            sharedID, *othercols=db.columns
            print(f'{filename}: joined at {sharedID}')
        else:
            print(f'{filename}: Missing shared key column')
            leftovers.append(file)
            continue
        # Merge cleaned up dfs
        db.drop_duplicates(subset=sharedID, keep='first', inplace=True)
        in_df=pd.merge(in_df, db[[sharedID, *othercols]],on=sharedID, how='left')
    return(in_df, leftovers)


def main():
    """
    Appends column-specific data from other .csv files to a designated .csv file
    """
    args=parse_args()

    # Set main db file
    db_csv=pd.read_csv(args.db)

    # Set directory of db files to merge
    db_files=glob_files(args.db_dir.rstrip('/'))

    # Merge each db_file to in_csv by shared key col, record unmerged files
    db_csv, db_leftovers=auto_merge(db_csv, db_files)
    
    # Retry with unmerged files, if applicable, then export .csv
    if db_leftovers:
        db_csv, remaining_leftovers=auto_merge(db_csv, db_leftovers)
        db_csv.to_csv(args.o, index=False, mode='w', header=True)
        if remaining_leftovers:
            remaining_filenames=[file.split('/')[-1] for file in remaining_leftovers]
            print(f'Remaining un-merged files: {remaining_filenames}')
    else:
        db_csv.to_csv(args.o, index=False, mode='w', header=True)

main()


In [ ]:
#@title Merge orthogonal dataset to .csv summary file (<1min)

import argparse
import pandas as pd

def parse_args():
    parser=argparse.ArgumentParser(prog='merge-to-db.py', conflict_handler='resolve')
    parser.add_argument('-i', type=str, required=True, help='=> path/to/infile.csv')
    parser.add_argument('-db', type=str, required=True, help='=> path/to/main_db')
    parser.add_argument('-keycolumn', type=str, required=True, help='=> key column to merge on')
    parser.add_argument('-o', type=str, required=True, help='=> path/to/merged_outfile.csv')
    return(parser.parse_args())

def main():
    """
    Appends column-specific data from a db.csv file to a designated .csv file
    """
    args=parse_args()

    # Set main input df to merge file
    in_df=pd.read_csv(args.i)

    # Set main db file and columns to merge using config (.ini)
    db_df=pd.read_csv(args.db)
    keycol=args.keycolumn
    # Re-order db_df and merge to input df on keycol val
    sID_col = db_df.pop(keycol)
    db_df.insert(0, sID_col.name, sID_col)
    sID, *othercols = db_df.columns
    in_df=pd.merge(in_df, db_df[[sID, *othercols]],on=keycol, how='left')

    # Hard-coded, this order doesn't need to change
    final_cols_order = ['sequenceID', 'Gene_Sym', 'description', 'AA_seqlength', 
        'start', 'count', 'Num_Unique', 'concat_sites', 'org_pvals', 'Non_hits', 
        'best_pval', 'human_site', 'pval_hg38', 'FUBAR_PSRs', 'Resource_Plate', 
        'Resource_Position', 'hORF_Length', 'PC1', 'Omega', 'calc_AF', 'log_calc_AF', 
        'human_hit', 'Ifn_u2', 'Ifn_u5', 'Ifn_d2', 'Ifn_d5']
    in_df=in_df.loc[:, final_cols_order]
    in_df.to_csv(args.o, index=False, mode='w', header=True)

main()

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

# if msa_mode == "custom":
#   print("Don't forget to cite your custom MSA generation method.")

# !zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png"
# files.download(f"{jobname}.result.zip")

# if save_to_google_drive == True and drive:
#   uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
#   uploaded.SetContentFile(f"{jobname}.result.zip")
#   uploaded.Upload()
#   print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")